In [5]:
!pip install torch torchaudio resampy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.7 MB/s eta 0:00:00a 0:00:01


In [14]:
!mc cp -r "s3/jujuschaff/vggish_multi_model.pt" "models/VGGish/vggish_multi_model.pt"

 0 B / ? ┃░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓┃
mc: <ERROR> Unable to prepare URL for copying. Unable to guess the type of copy operation.



In [9]:
import sys
sys.path.insert(0, "models/VGGish")
import argparse
import os
import random

import numpy as np
import torch
import torchaudio.functional as F
import config

from utils import download_dataset_if_needed

from models.VGGish.vggish_multichannel import VGGishMulti

import pandas as pd
import numpy as np

import os

sys.path.insert(0, 'datasets/')

from datasets.dataset import Dataset
from datasets import livingroom
from datasets.roomsetup import RoomSetup
import matplotlib.pyplot as plt

import librosa

from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split

from utils import download_dataset_if_needed, get_dataset

In [10]:
download_dataset_if_needed()

path: ./LivingRoom_preprocessed_hack already exist, ignorign dataset downloading


In [11]:
DATASET_PATH = "LivingRoom_preprocessed_hack"
empty_dataset = get_dataset(None, "deconvolved.npy", os.path.join(DATASET_PATH, "Empty"))
human1_dataset = get_dataset("centroid.npy", "deconvoled_trim.npy", os.path.join(DATASET_PATH, "Human1"))
human2_dataset = get_dataset("centroid.npy", "deconvoled_trim.npy", os.path.join(DATASET_PATH, "Human2"))

Shape of RIRs:
(100, 10, 667200)
Shape of Centroid:
(1000, 2)
Shape of RIRs:
(1000, 4, 667200)
Shape of Centroid:
(104, 2)
Shape of RIRs:
(104, 4, 667200)


In [12]:
save_path = "models/VGGish/vggish_multi_model.pt"
error_path ="models/VGGish/vggish_multi_errors/"

# Choose if you want to use an already saved model
#vggish_multi = VGGishMulti(save_path=save_path, error_path=error_path)
vggish_multi = VGGishMulti(saved_model=save_path, save_path=save_path, error_path=error_path)

FileNotFoundError: [Errno 2] No such file or directory: 'models/VGGish/vggish_multi_model.pt'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(human1_dataset[1], human1_dataset[2], test_size=0.2, random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(human1_dataset[1], human1_dataset[2], test_size=0.25, random_state=42)

In [ ]:
# Train the VGGishMulti model
# vggish_multi.fit(X_train, y_train, X_valid, y_valid, num_epochs=10)

In [ ]:
result = vggish_multi.predict(X_test, y_test)

In [ ]:
dist = np.linalg.norm(y_pred - y_test,axis=1)

In [ ]:
avg = np.average(dist)
std = np.std(dist)
print("Distance difference from real position in millimeters:", avg)
print("Standard deviation in millimeters:", std)

In [ ]:
human1_dataset[0].room_setup.plot_prediction(y_pred, y_test)

In [ ]:
human1_dataset[0].room_setup.plot_heatmap(y_pred, y_test, avg)